# Winner of the tournament
We add columns to tournamen_info_df that have:
- The winner of the tournament
- If the winner came from losers
- The ratings of players playing in the top 8

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import datetime 

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, cohen_kappa_score
from sklearn.model_selection import train_test_split  # Correct import

import sqlite3
import sys
import time
import tqdm
from tqdm.auto import tqdm
import pickle
import joblib
import os

if os.path.exists('/workspace/data_2'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data_2/'
else:
    data_path = '../data/'


### Load Data


In [95]:
sets_df = pd.read_pickle(data_path + 'full_dataset_df.pkl')

sets_df.head(3)



,key_x,game,tournament_key,winner_id,loser_id,p1_id,p2_id,p1_score,p2_score,valid_score,...,p1/m1/m1_alt2_updates,p2/m1/m1_alt2_elo,p2/m1/m1_alt2_rd,p2/m1/m1_alt2_updates,p1/m1_alt3_elo,p1/m1_alt3_rd,p1/m1_alt3_updates,p2/m1_alt3_elo,p2/m1_alt3_rd,p2/m1_alt3_updates
0,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,Chillin,Chillin,5620,1,3,True,...,0.0,1951.007767,92.862909,6.0,1500.0,350.0,0.0,1613.001275,70.600790,5.0
1,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,Chillin,Chillin,5620,1,3,True,...,0.0,1951.007767,92.862909,6.0,1500.0,350.0,0.0,1613.001275,70.600790,5.0
2,104675844,melee,mdva-invitational-2017-(challonge-mirror),Aglet,15634,Aglet,15634,3,2,True,...,0.0,2217.315033,196.486995,4.0,1500.0,350.0,0.0,1637.162591,77.931512,5.0


In [96]:
for i, col in enumerate(sets_df.columns):
    print(i, col)

0 key_x
1 game
2 tournament_key
3 winner_id
4 loser_id
5 p1_id
6 p2_id
7 p1_score
8 p2_score
9 valid_score
10 best_of
11 location_names
12 bracket_name
13 bracket_order
14 set_order
15 game_data
16 top_8
17 top_8_location_names
18 valid_top_8_bracket
19 top_8_bracket_location_names
20 major
21 key_y
22 start
23 end
24 start_week
25 p1_characters
26 p2_characters
27 p1_consistent
28 p2_consistent
29 matchup_strings
30 end_week
31 players_have_history
32 (p1/p2)_sorted
33 (p1/p2)_was_sorted
34 results_sorted
35 results
36 matchup_1
37 matchup_2
38 matchup_3
39 matchup_4
40 matchup_5
41 matchup_6
42 matchup_7
43 matchup_8
44 matchup_9
45 matchup_10
46 winner
47 p1_default_elo
48 p2_default_elo
49 p1_default_rd
50 p2_default_rd
51 p1_default_updates
52 p2_default_updates
53 start_index
54 start_date
55 p1_fox_count
56 p1_falco_count
57 p1_marth_count
58 p1_sheik_count
59 p1_captainfalcon_count
60 p1_jigglypuff_count
61 p1_peach_count
62 p1_luigi_count
63 p1_samus_count
64 p1_ganondorf_coun

In [97]:
tournament_info_df = pd.read_pickle(data_path + 'top_8_tournament_previous_sets_and_results_df')
print(tournament_info_df.columns)
tournament_info_df.head()

Index(['game', 'key', 'cleaned_name', 'source', 'tournament_name',
       'tournament_event', 'season', 'rank', 'start', 'end', 'country',
       'state', 'city', 'entrants', 'placings', 'losses', 'bracket_types',
       'online', 'lat', 'lng', 'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2',
       'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2',
       'LN_A_p1_non_top_8_sets', 'LN_A_p2_non_top_8_sets',
       'LN_B_p1_non_top_8_sets', 'LN_B_p2_non_top_8_sets',
       'WSF_A_p1_non_top_8_sets', 'WSF_A_p2_non_top_8_sets',
       'WSF_B_p1_non_top_8_sets', 'WSF_B_p2_non_top_8_sets'],
      dtype='object')


,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,...,WSF_B_p1,WSF_B_p2,LN_A_p1_non_top_8_sets,LN_A_p2_non_top_8_sets,LN_B_p1_non_top_8_sets,LN_B_p2_non_top_8_sets,WSF_A_p1_non_top_8_sets,WSF_A_p2_non_top_8_sets,WSF_B_p1_non_top_8_sets,WSF_B_p2_non_top_8_sets
0,melee,mdva-invitational-2017-(challonge-mirror),MDVA Invitational 2017 (Challonge Mirror),challonge,https://challonge.com/mdva_invitational_2017,,17,,2017-11-26 08:05:11,2017-11-26 08:48:09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,melee,s@sh7,S@SH7,challonge,https://challonge.com/sash7,,17,,2017-06-13 10:27:01,2017-06-13 10:27:01,...,Ginger,1008,"[(84, True), (110, True), (123, False), (210, ...","[(77, True), (107, True), (122, False), (209, ...","[(85, True), (111, True), (124, True), (130, F...","[(92, True), (114, True), (125, False), (204, ...","[(81, True), (109, True), (123, True), (130, T...","[(73, True), (105, True), (121, True), (129, T...","[(97, True), (117, True), (127, True), (132, T...","[(89, True), (113, True), (125, True), (131, T..."
2,melee,slippi-champions-league-week-1__melee-singles,Slippi Champions League Week 1,pgstats,slippi-champions-league-week-1,melee-singles,20,,2020-10-11 14:00:00,2020-10-11 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,melee,slippi-champions-league-week-2__melee-singles,Slippi Champions League Week 2,pgstats,slippi-champions-league-week-2,melee-singles,20,,2020-10-18 14:00:00,2020-10-18 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,melee,slippi-champions-league-week-3__melee-singles,Slippi Champions League Week 3,pgstats,slippi-champions-league-week-3,melee-singles,20,,2020-10-25 14:00:00,2020-10-25 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Winner of the tournament

In [132]:
valid_sets = sets_df[sets_df['valid_top_8_bracket']==True]

# Identify tournaments with 'GFR'
GFR_tournament_keys = valid_sets.loc[(valid_sets['top_8_bracket_location_names'] == 'GFR'), 'tournament_key'].copy()

# Filter 'GF' tournaments that do not have a corresponding 'GFR'
GF_tournament_results_df = valid_sets.loc[
    (valid_sets['top_8_bracket_location_names'] == 'GF') & (~valid_sets['tournament_key'].isin(GFR_tournament_keys)),
    ['tournament_key', 'winner_id']
].copy()

GF_tournament_results_df['GFR'] = False

# Filter 'GFR' tournament results
GFR_tournament_results_df = valid_sets.loc[
    valid_sets['top_8_bracket_location_names'] == 'GFR',
    ['tournament_key', 'winner_id']
]
GFR_tournament_results_df['GFR'] = True

# Concatenate the results
tournament_results_df = pd.concat([GF_tournament_results_df, GFR_tournament_results_df], ignore_index=False)

winners_finals_players_df = valid_sets[(valid_sets['top_8_location_names']=='WF')][['tournament_key', 'p1_id', 'p2_id']]

# Create a mapping of tournament_key to all players in losers finals
winners_finals_players_df = (
    winners_finals_players_df
    .groupby('tournament_key')[['p1_id', 'p2_id']]
    .apply(lambda df: set(df.values.flatten()))
    .to_dict()
)

# Update 'from_WF' column based on whether the winner_id is in the losers finals players
tournament_results_df['from_WF'] = tournament_results_df.apply(
    lambda row: row['winner_id'] in winners_finals_players_df.get(row['tournament_key'], set()),
    axis=1
)

top_8_player_columns = ['key', 'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2',
                        'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2']

# Perform the merge and retain the index of `tournament_results_df`
tournament_results_df = pd.merge(
    tournament_results_df,
    tournament_info_df[top_8_player_columns],
    how='left',
    left_on='tournament_key',
    right_on='key'
)

# Drop the 'key' column
tournament_results_df = tournament_results_df.drop(columns=['key'])

# Preserve the original index of `tournament_results_df`
tournament_results_df.index = tournament_results_df.index

# Update 'from_winners' column based on whether the winner_id is in the WSF player columns
tournament_results_df['from_winners_side'] = tournament_results_df.apply(
    lambda row: row['winner_id'] in [
        row['WSF_A_p1'], row['WSF_A_p2'], row['WSF_B_p1'], row['WSF_B_p2']
    ],
    axis=1
)

# View the results
tournament_results_df.head()


,tournament_key,winner_id,GFR,from_WF,LN_A_p1,LN_A_p2,LN_B_p1,LN_B_p2,WSF_A_p1,WSF_A_p2,WSF_B_p1,WSF_B_p2,from_winners_side
0,s@sh7,Mew2king (unpaid),False,True,math,lain,tm,Bbbbbbbbbexic,kjh,Mew2king (unpaid),Ginger,1008,True
1,httpsparagonchallongecomla_2015_melee_singles,1000,False,True,1008,1023,13932,1004,1000,16342,1003,4465,True
2,httpsdl4-5challongecomdl45meleeprobracket,6189,False,True,Hack,Vdogg,4215,Thomas,6103,6189,4465,12870,True
3,httpsbeastsmashchallongecomb5msb,4465,False,True,Prof,48789,12870,5956,19573,1000,4465,6189,True
4,httpsparagonchallongecomorlando_2015_melee_sin...,1004,False,True,16342,4465,1008,1013,1004,6189,1003,15990,True


In [133]:
print(f"The winner of the tournament came from winners side {tournament_results_df['from_winners'].sum() / tournament_results_df.shape[0]:.0%} of the time.")
print(f"The winner of the tournament came from winners finals {tournament_results_df['from_WF'].sum() / tournament_results_df.shape[0]:.0%} of the time.")
print(f"A grand finals reset happened {tournament_results_df['GFR'].sum() / tournament_results_df.shape[0]:.0%} of the time.")
print(f"Given that a grand final reset occured, the player coming from the winners finals won {tournament_results_df[tournament_results_df['GFR']==True]['from_WF'].sum() / tournament_results_df['GFR'].sum():.0%} of the time.")
tournament_results_df.head()

KeyError: 'from_winners'

In [134]:
top_8_player_columns = ['LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2',
                        'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2']


In [135]:
# Define the top 8 player columns and their corresponding bracket locations
top_8_player_columns = ['LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2',
                        'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2']
bracket_locations = ['LN_A', 'LN_B', 'WSF_A', 'WSF_B']

# Prepare a copy of tournament_results_df
result_df = tournament_results_df.copy()

# Iterate through the top 8 player columns and merge the corresponding ELO data
for bracket_location in bracket_locations:
    # Filter valid sets for the current bracket location
    relevant_sets = valid_sets[valid_sets['top_8_bracket_location_names'] == bracket_location][
        ['tournament_key', 'p1_default_elo', 'p2_default_elo']
    ]
    
    # Rename columns to ensure uniqueness
    relevant_sets.rename(columns={
        'p1_default_elo': f'{bracket_location}_p1_elo',
        'p2_default_elo': f'{bracket_location}_p2_elo'
    }, inplace=True)
    
    # Merge the ELO data with the result_df
    result_df = pd.merge(
        result_df,
        relevant_sets,
        on='tournament_key',
        how='left'
    )
    
result_df = result_df.loc[:, ~result_df.columns.duplicated()]


# Check for and drop duplicate columns


# View the resulting DataFrame
print(result_df.head().to_string())


                                       tournament_key          winner_id    GFR  from_WF LN_A_p1 LN_A_p2 LN_B_p1        LN_B_p2 WSF_A_p1           WSF_A_p2 WSF_B_p1 WSF_B_p2  from_winners_side  LN_A_p1_elo  LN_A_p2_elo  LN_B_p1_elo  LN_B_p2_elo  WSF_A_p1_elo  WSF_A_p2_elo  WSF_B_p1_elo  WSF_B_p2_elo
0                                               s@sh7  Mew2king (unpaid)  False     True    math    lain      tm  Bbbbbbbbbexic      kjh  Mew2king (unpaid)   Ginger     1008               True  1500.000000  1500.000000  1669.343857  1500.000000   1500.000000   1500.000000   1643.796527   2059.466548
1       httpsparagonchallongecomla_2015_melee_singles               1000  False     True    1008    1023   13932           1004     1000              16342     1003     4465               True  1878.237223  1802.177215  1823.077409  2001.144322   1950.348732   1888.611974   1876.719754   1964.348729
2           httpsdl4-5challongecomdl45meleeprobracket               6189  False     True    Hack 

In [136]:
player_col = result_df.columns[4:12]
elo_col = result_df.columns[-8:]

def extract_first_value(cell):
    if isinstance(cell, list):
        return cell[0]  # Extract the first value from the list
    return cell  # Return the value if it's not a list

# Create a copy of the relevant columns to work with, without modifying result_df
processed_df = result_df.copy()

# Process ELO columns
for col in elo_col:
    if processed_df[col].apply(lambda x: isinstance(x, list)).any():  # Check if lists exist
        processed_df[col] = processed_df[col].apply(extract_first_value)
    if not pd.api.types.is_numeric_dtype(processed_df[col]):  # Check if numeric conversion is needed
        processed_df[col] = pd.to_numeric(processed_df[col], errors='coerce')  # Convert to numeric

# Convert ELOs and players to NumPy arrays
elos = processed_df[elo_col].to_numpy()
sorted_elos = np.argsort(-elos, axis=1)  # Indices of ELOs sorted in descending order
players = processed_df[player_col].to_numpy()
winner = processed_df['winner_id'].to_numpy()

# Create a mask where the winner appears in the players array for each row
winner_mask = np.array([row == win for row, win in zip(players, winner)])

# Get the rank of the winner based on sorted elos
winner_elo_ranks = np.array([
    np.where(sorted_elos[row_idx] == np.argmax(mask))[0][0] + 1  # Find the rank of the winner (1-based)
    for row_idx, mask in enumerate(winner_mask)
])

# Create a new DataFrame for results, leaving result_df unchanged
result_with_ranks = processed_df.copy()
result_with_ranks['elo_rank_of_winner'] = winner_elo_ranks

# Count the occurrences of each rank
rank_counts = result_with_ranks[['elo_rank_of_winner']].value_counts()

# Print the rank counts
print(rank_counts)


elo_rank_of_winner
1                     19449
2                      5723
3                      2346
4                      1233
5                       691
6                       444
7                       330
8                       273
Name: count, dtype: int64


In [ ]:
player_col = result_df.columns[4:12]
elo_col = result_df.columns[-8:]
winners_player_col = result_df.columns[8:12]
winners_elo_col = result_df.columns[-4:]

def extract_first_value(cell):
    if isinstance(cell, list):
        return cell[0]  # Extract the first value from the list
    return cell  # Return the value if it's not a list

# Create a copy of the relevant columns to work with, without modifying result_df
processed_df = result_df.copy()

# Process all ELO columns
for col in list(elo_col) + list(winners_elo_col):
    if processed_df[col].apply(lambda x: isinstance(x, list)).any():  # Check if lists exist
        processed_df[col] = processed_df[col].apply(extract_first_value)
    if not pd.api.types.is_numeric_dtype(processed_df[col]):  # Check if numeric conversion is needed
        processed_df[col] = pd.to_numeric(processed_df[col], errors='coerce')  # Convert to numeric

# Convert data to NumPy arrays
elos = processed_df[elo_col].to_numpy()
players = processed_df[player_col].to_numpy()
winners_elos = processed_df[winners_elo_col].to_numpy()
winners_players = processed_df[winners_player_col].to_numpy()
winner = processed_df['winner_id'].to_numpy()
from_winners = processed_df['from_winners'].to_numpy()

# Sort ELOs in descending order for all players and winners
sorted_elos = np.argsort(-elos, axis=1)
sorted_winners_elos = np.argsort(-winners_elos, axis=1)

# Create masks where the winner appears in the players and winners arrays
winner_mask = np.array([row == win for row, win in zip(players, winner)])
winner_from_winners_mask = np.array([row == win for row, win in zip(winners_players, winner)])

# Get the rank of the winner based on all sorted elos
winner_elo_ranks = np.array([
    np.where(sorted_elos[row_idx] == np.argmax(mask))[0][0] + 1  # Find the rank of the winner (1-based)
    for row_idx, mask in enumerate(winner_mask)
])

# Get the rank of the winner among players from winners' side
winner_rank_among_winners = np.array([
    (np.where(sorted_winners_elos[row_idx] == np.argmax(mask))[0][0] + 1) if from_winners[row_idx] else None
    for row_idx, mask in enumerate(winner_from_winners_mask)
])

# Create a new DataFrame for results, leaving result_df unchanged
result_with_ranks = processed_df.copy()
result_with_ranks['elo_rank_of_winner'] = winner_elo_ranks
result_with_ranks['elo_rank_among_winners'] = winner_rank_among_winners

# Count the occurrences of each rank
rank_counts = result_with_ranks[['elo_rank_of_winner']].value_counts()

# Print the rank counts and preview the result
rank_counts


elo_rank_of_winner
1                     19449
2                      5723
3                      2346
4                      1233
5                       691
6                       444
7                       330
8                       273
Name: count, dtype: int64
   elo_rank_of_winner elo_rank_among_winners
0                   8                      4
1                   3                      2
2                   1                      1
3                   7                      3
4                   6                      2
